# Example of DOV search methods for soil data (bodemgegevens)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_bodem.ipynb)

## Use cases explained below
* Introduction to the bodem-objects
* Get bodemsites in a bounding box
* Get bodemlocaties with specific properties
* Get all direct and indirect bodemobservaties linked to a bodemlocatie
* Get all bodemobservaties in a bodemmonster
* Find all bodemlocaties where observations exist for organic carbon percentage in East-Flanders between 0 and 30 cm deep
* Calculate carbon stock in Ghent in the layer 0 - 23 cm

In [14]:
%matplotlib inline

import inspect, sys
import warnings; warnings.simplefilter('ignore')

In [15]:
# check pydov path
import pydov

## Get information about the datatype 'Bodemlocatie'

Other datatypes are also possible:
* Bodemsite: BodemsiteSearch
* Bodemmonster: BodemmonsterSearch
* Bodemobservatie: BodemobservatieSearch

In [16]:
from pydov.search.bodemlocatie import BodemlocatieSearch
bodemlocatie = BodemlocatieSearch()

A description is provided for the 'Bodemlocatie' datatype:

In [17]:
bodemlocatie.get_description()

"Een bodemlocatie is ofwel een profielput of een boring. Een boring is altijd één puntlocatie (x,y,z) en een profielput heeft minimum één en maximum twee puntlocaties (begin- en eindpunt van de profielput). \r\nEen profielput is een uitgegraven put in de bodem waarin profielbeschrijvingen, monsternames of bodemobservaties worden uitgevoerd. Een profielbeschrijving is een waarneming van bodemhorizonten en/of bodemlagen in een uitgegraven profielput. Een bodemhorizont is een visueel te onderscheiden deel van de bodem dat ontstaan is door omzetting van het moedermateriaal door pedogenetische processen of door het afzetten van organisch materiaal. Een bodemhorizont heeft voor de meeste bodemvariabelen homogene morfologische en analytische karakteristieken. Een bodemlaag daarentegen is ontstaan door niet-pedogenetische processen. Aan de hand van een profielput krijg je een beeld van de bodemkundige opbouw.\r\nEen boring is het resultaat van het boren in de ondergrond met verwijdering van bo

The different fields that are available for objects of the 'Bodemlocatie' datatype can be requested with the get_fields() method:

In [18]:
fields = bodemlocatie.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

naam
pkey_bodemlocatie
type
rapport_bodemlocatie
profielbeschrijving
waarnemingsdatum
doel
x
y
mv_mtaw
Auteurs
Aantal_classificaties
Aantal_opbouwen
erfgoed
Aantal_observaties
Aantal_monsters
bodemstreek
Bodemsite
pkey_bodemsite
Opdrachten
invoerdatum
educatieve_waarde


You can get more information of a field by requesting it from the fields dictionary:
* *name*: name of the field
* *definition*: definition of this field
* *cost*: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
* *notnull*: whether the field is mandatory or not
* *type*: datatype of the values of this field

In [19]:
fields['type']

{'name': 'type',
 'definition': 'Het type van de bodemlocatie: boring of profielput.',
 'type': 'string',
 'notnull': False,
 'query': True,
 'cost': 1,
 'values': {'boring': 'Een boring is het resultaat van het boren in de ondergrond met verwijdering van bodem door middel van een gereedschap in de vorm van een holle buis. Aan de hand van dit opgeboorde bodemmateriaal worden bodembeschrijvingen, bodemobservaties en monsternames uitgevoerd.',
  'profielput': 'Een profielput is een uitgegraven put in de bodem waarin profielbeschrijvingen, monsternames of bodemobservaties worden uitgevoerd. Een profielbeschrijving is een waarneming van bodemhorizonten en/of bodemlagen in een uitgegraven profielput. Een bodemhorizont is een visueel te onderscheiden deel van de bodem dat ontstaan is door omzetting van het moedermateriaal door pedogenetische processen of door het afzetten van organisch materiaal. Een bodemhorizont heeft voor de meeste bodemvariabelen homogene morfologische en analytische kar

Optionally, if the values of the field have a specific domain the possible values are listed as *values*:

In [20]:
fields['type']['values']

{'boring': 'Een boring is het resultaat van het boren in de ondergrond met verwijdering van bodem door middel van een gereedschap in de vorm van een holle buis. Aan de hand van dit opgeboorde bodemmateriaal worden bodembeschrijvingen, bodemobservaties en monsternames uitgevoerd.',
 'profielput': 'Een profielput is een uitgegraven put in de bodem waarin profielbeschrijvingen, monsternames of bodemobservaties worden uitgevoerd. Een profielbeschrijving is een waarneming van bodemhorizonten en/of bodemlagen in een uitgegraven profielput. Een bodemhorizont is een visueel te onderscheiden deel van de bodem dat ontstaan is door omzetting van het moedermateriaal door pedogenetische processen of door het afzetten van organisch materiaal. Een bodemhorizont heeft voor de meeste bodemvariabelen homogene morfologische en analytische karakteristieken. Een bodemlaag daarentegen is ontstaan door niet-pedogenetische processen. Aan de hand van een profielput krijg je een beeld van de bodemkundige opbouw

## Example use cases

### Get bodemsites in a bounding box

Get data for all the bodemsites that are geographically located completely within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

The same methods can be used for other bodem objects.

In [21]:
from pydov.search.bodemsite import BodemsiteSearch
bodemsite = BodemsiteSearch()

In [22]:
from pydov.util.location import Within, Box

df = bodemsite.search(location=Within(Box(148000, 160800, 160000, 169500)))
df.head()

[000/009] c........


,pkey_bodemsite,naam,waarnemingsdatum,beschrijving,invoerdatum
0,https://www.dov.vlaanderen.be/data/bodemsite/2...,CGAR_Pafed294_Plot,2019-10-29,Park. Deze bodemsite beschrijft de geometrie w...,2020-07-27
1,https://www.dov.vlaanderen.be/data/bodemsite/2...,CGAR_Pafed295_Plot,2019-10-29,Park,2020-07-27
2,https://www.dov.vlaanderen.be/data/bodemsite/2...,Zoniënwoud_Groenendaal,NaN,"Zachte helling op plateau, Bodem nooit onder l...",NaN
3,https://www.dov.vlaanderen.be/data/bodemsite/2...,Zoniënwoud_Rood Klooster,NaN,Zeer zachte helling op plateau,NaN
4,https://www.dov.vlaanderen.be/data/bodemsite/1...,Zoniënwoud_Tertiair,1983-01-01,NNW gerichte helling van een 10% langs vallei ...,NaN


The dataframe contains a list of bodemsites. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_bodemsite* field one can request the details of this bodemsite in a webbrowser:

In [23]:
for pkey_bodemsite in set(df.pkey_bodemsite):
    print(pkey_bodemsite)

https://www.dov.vlaanderen.be/data/bodemsite/2021-000310
https://www.dov.vlaanderen.be/data/bodemsite/2017-000210
https://www.dov.vlaanderen.be/data/bodemsite/2017-000229
https://www.dov.vlaanderen.be/data/bodemsite/1983-000216
https://www.dov.vlaanderen.be/data/bodemsite/2021-000309
https://www.dov.vlaanderen.be/data/bodemsite/2017-000215
https://www.dov.vlaanderen.be/data/bodemsite/2017-000305
https://www.dov.vlaanderen.be/data/bodemsite/2011-000247
https://www.dov.vlaanderen.be/data/bodemsite/2017-000306


### Get bodemlocaties with specific properties

Next to querying bodem objects based on their geographic location within a bounding box, we can also search for bodem objects matching a specific set of properties. 
The same methods can be used for all bodem objects.
For this we can build a query using a combination of the 'Bodemlocatie' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [24]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes'], inspect.isclass) if 'Property' in i]

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In this example we build a query using the *PropertyIsEqualTo* operator to find all bodemlocaties with bodemstreek 'zandstreek'.
We use *max_features=10* to limit the results to 10.

In [25]:
from owslib.fes import PropertyIsEqualTo

query = PropertyIsEqualTo(propertyname='bodemstreek',
                          literal='Zandstreek')
df = bodemlocatie.search(query=query, max_features=10)

df.head()

[000/010] c.........


,pkey_bodemlocatie,pkey_bodemsite,naam,type,waarnemingsdatum,doel,x,y,mv_mtaw,erfgoed,bodemstreek,invoerdatum,educatieve_waarde
0,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,ARCH_2017C381_WP0001BP01,profielput,2017-03-27,archeologische proefputten ifv prehistorische ...,100951.02,197304.89,6.11,False,Zandstreek,2021-05-06,NaN
1,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,ARCH_2018D174_1,profielput,2018-11-07,archeologische proefsleuven en proefputten,136567.44,209067.73,13.17,False,Zandstreek,2021-06-17,NaN
2,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,ARCH_2018D174_2,profielput,2018-11-07,archeologische proefsleuven en proefputten,136520.06,209040.88,13.75,False,Zandstreek,2021-06-17,NaN
3,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,ARCH_2018D174_3,profielput,2018-11-07,archeologische proefsleuven en proefputten,136458.47,209006.77,13.41,False,Zandstreek,2021-06-17,NaN
4,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,ARCH_2021L196_463.1,profielput,2021-12-21,archeologische proefsleuven en proefputten,119075.97,201394.21,5.75,False,Zandstreek,2022-01-07,NaN


Once again we can use the *pkey_bodemlocatie* as a permanent link to the information of these bodemlocaties:

In [26]:
for pkey_bodemlocatie in set(df.pkey_bodemlocatie):
    print(pkey_bodemlocatie)

https://www.dov.vlaanderen.be/data/bodemlocatie/2021-024152
https://www.dov.vlaanderen.be/data/bodemlocatie/2018-022979
https://www.dov.vlaanderen.be/data/bodemlocatie/2018-022978
https://www.dov.vlaanderen.be/data/bodemlocatie/2018-022980
https://www.dov.vlaanderen.be/data/bodemlocatie/2021-024094
https://www.dov.vlaanderen.be/data/bodemlocatie/2017-018199
https://www.dov.vlaanderen.be/data/bodemlocatie/2010-000014
https://www.dov.vlaanderen.be/data/bodemlocatie/2012-000041
https://www.dov.vlaanderen.be/data/bodemlocatie/2007-000102
https://www.dov.vlaanderen.be/data/bodemlocatie/2021-024151


### Get all direct and indirect bodemobservaties in bodemlocatie

Get all bodemobservaties in a specific bodemlocatie.
Direct means bodemobservaties directly linked with a bodemlocatie.
Indirect means bodemobservaties linked with child-objects of the bodemlocatie, like bodemmonsters.

In [27]:
from pydov.search.bodemobservatie import BodemobservatieSearch
from pydov.search.bodemlocatie import BodemlocatieSearch
bodemobservatie = BodemobservatieSearch()
bodemlocatie = BodemlocatieSearch()

In [28]:
from owslib.fes import PropertyIsEqualTo
from pydov.util.query import Join

bodemlocaties = bodemlocatie.search(query=PropertyIsEqualTo(propertyname='naam', literal='VMM_INF_52'), 
                                   return_fields=('pkey_bodemlocatie',))

bodemobservaties = bodemobservatie.search(query=Join(bodemlocaties, 'pkey_bodemlocatie'))
bodemobservaties.head()

[000/021] .....................


,pkey_bodemobservatie,pkey_bodemlocatie,pkey_parent,x,y,mv_mtaw,diepte_van_cm,diepte_tot_cm,observatiedatum,invoerdatum,parametergroep,parameter,detectie,waarde,eenheid,veld_labo,methode,betrouwbaarheid
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,129586.0,182459.0,30.7,40.0,85.0,2015-09-08,2019-02-07,Bodem_fysisch_vocht,Ksat,NaN,0.0000154321,m/s,veld,Ksat_Soakaway_methode,goed
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,129586.0,182459.0,30.7,40.0,85.0,2015-09-08,2019-02-07,Bodem_fysisch_vocht,Ksat,NaN,0.0000029437,m/s,veld,Ksat Enkele-ring-methode,goed
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,129586.0,182459.0,30.7,40.0,85.0,2015-09-08,2019-02-07,Bodem_fysisch_vocht,Ksat,NaN,0.0000100245,m/s,veld,Ksat Porchet-methode,goed
3,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,129586.0,182459.0,30.7,40.0,85.0,2015-09-08,2019-02-07,Bodem_fysisch_vocht,Ksat,NaN,0.0000036111,m/s,veld,Ksat Enkele-ring-methode,goed
4,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,129586.0,182459.0,30.7,40.0,85.0,2015-09-08,2019-02-07,Bodem_fysisch_vocht,Ksat,NaN,0.0000146735,m/s,veld,Ksat_Soakaway_methode,goed


### Get all bodemobservaties in a bodemmonster

Get all bodemobservaties linked with a bodemmonster

In [29]:
from pydov.search.bodemmonster import BodemmonsterSearch
bodemmonster = BodemmonsterSearch()

bodemmonsters = bodemmonster.search(query=PropertyIsEqualTo(propertyname = 'identificatie', literal='A0057359'),
                                   return_fields=('pkey_bodemmonster',))

bodemobservaties = bodemobservatie.search(query=Join(bodemmonsters, on = 'pkey_parent', using='pkey_bodemmonster'))
bodemobservaties.head()

[000/003] ccc


,pkey_bodemobservatie,pkey_bodemlocatie,pkey_parent,x,y,mv_mtaw,diepte_van_cm,diepte_tot_cm,observatiedatum,invoerdatum,parametergroep,parameter,detectie,waarde,eenheid,veld_labo,methode,betrouwbaarheid
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,129586.0,182459.0,30.7,40.0,85.0,2015-08-27,2019-02-07,Bodem_fysisch_textuur,Textuur - granulometrisch - klassen bodemkarte...,NaN,E - Klei,-,labo,Textuur pipetmethode Robinson-Köhn,goed
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,129586.0,182459.0,30.7,40.0,85.0,2015-08-27,2019-02-07,Bodem_fysisch_textuur,Textuur - handmatig - klassen bodemkartering,NaN,A - Leem,-,labo,Textuur manuele schatting (palpatie),goed
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,129586.0,182459.0,30.7,40.0,85.0,2015-09-08,2019-02-19,Bodem_fysisch_textuur,Textuurfracties,NaN,Fractiemetingen,%,labo,Textuur pipetmethode Robinson-Köhn,goed


### Find all soil locations with a given soil classification

Get all soil locations with a given soil classification:

In [30]:
from owslib.fes import PropertyIsEqualTo
from pydov.util.query import Join

from pydov.search.bodemclassificatie import BodemclassificatieSearch
from pydov.search.bodemlocatie import BodemlocatieSearch

bodemclassificatie = BodemclassificatieSearch()
bl_Scbz = bodemclassificatie.search(query=PropertyIsEqualTo('bodemtype', 'Scbz'), return_fields=['pkey_bodemlocatie'])

bodemlocatie = BodemlocatieSearch()
bl = bodemlocatie.search(query=Join(bl_Scbz, 'pkey_bodemlocatie'))
bl.head()

[000/010] cccccccccc


,pkey_bodemlocatie,pkey_bodemsite,naam,type,waarnemingsdatum,doel,x,y,mv_mtaw,erfgoed,bodemstreek,invoerdatum,educatieve_waarde
0,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemsite/2...,HAAST_16,profielput,2015-04-16,archeologische landschappelijke profielputten,248905.67,200391.29,32.88,True,Kempen,2015-04-16,ZEER
1,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,KART_PROF_071E/28,profielput,1960-09-30,bodemprofielen en oppervlaktemonsters karterin...,129724.00,186794.00,7.00,False,Zandstreek,2019-10-11,OK
2,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,KART_PROF_071E/24,profielput,1960-09-29,bodemprofielen en oppervlaktemonsters karterin...,128409.00,185764.00,5.00,False,Zandleemstreek,2019-10-11,OK
3,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,KART_PROF_071E/21,profielput,1960-09-28,bodemprofielen en oppervlaktemonsters karterin...,127729.00,185179.00,5.00,False,Zandleemstreek,2019-10-11,OK
4,https://www.dov.vlaanderen.be/data/bodemlocati...,NaN,KART_OPP_053W/030,boring,1968-01-01,bodemprofielen en oppervlaktemonsters karterin...,73845.00,195091.00,-9999.00,False,Kunstmatige gronden,2019-10-12,OK


We can also get their observations:

In [31]:
from pydov.search.bodemobservatie import BodemobservatieSearch

bodemobservatie = BodemobservatieSearch()
obs = bodemobservatie.search(query=Join(bl_Scbz, 'pkey_bodemlocatie'), max_features=10)
obs.head()

[000/010] cccccccccc


,pkey_bodemobservatie,pkey_bodemlocatie,pkey_parent,x,y,mv_mtaw,diepte_van_cm,diepte_tot_cm,observatiedatum,invoerdatum,parametergroep,parameter,detectie,waarde,eenheid,veld_labo,methode,betrouwbaarheid
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,127729.0,185179.0,5.0,0.0,22.0,1960-09-28,NaN,Bodem_kleur,Kleur volgens Munsell - HUE_letters (kleur1),NaN,YR,NaN,veld,NaN,onbekend
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,127729.0,185179.0,5.0,0.0,22.0,1960-09-28,NaN,Bodem_fysisch_vocht,Vochtgehalte gradatie,NaN,4 - vochtig,NaN,veld,NaN,onbekend
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,127729.0,185179.0,5.0,22.0,44.0,1960-09-28,NaN,Bodem_fysisch_textuur,Textuur - handmatig - gedetailleerd,NaN,S - Lemig zand,NaN,veld,NaN,onbekend
3,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,127729.0,185179.0,5.0,62.0,85.0,1960-09-28,NaN,Bodem_fysisch_vocht,Gley roest - aantal,NaN,Ra2 - Matig veel (2-20 %) - Tamelijk/ Goed roe...,NaN,veld,NaN,onbekend
4,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemdiepte...,127729.0,185179.0,5.0,85.0,125.0,1960-09-28,NaN,Bodem_fysisch_vocht,Gley roest - contrast,NaN,Rc2 - Duidelijk,NaN,veld,NaN,onbekend


### Find all bodemlocaties where observations exist for organic carbon percentage in East-Flanders between 0 and 30 cm deep

Get boundaries of East-Flanders by using a WFS

In [32]:
from owslib.etree import etree
from owslib.wfs import WebFeatureService
from pydov.util.location import (
    GmlFilter,
    Within,
)

provinciegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

provincie_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Oost-Vlaanderen')
provincie_poly = provinciegrenzen.getfeature(
    typename='VRBG:Refprv',
    filter=etree.tostring(provincie_filter.toXML()).decode("utf8")).read()

Get bodemobservaties in East-Flanders with the requested properties

In [33]:
from owslib.fes import PropertyIsEqualTo
from owslib.fes import And

from pydov.search.bodemobservatie import BodemobservatieSearch

bodemobservatie = BodemobservatieSearch()

# Select only layers with the boundaries 10-30
bodemobservaties = bodemobservatie.search(
        location=GmlFilter(provincie_poly, Within),
        query=And([
            PropertyIsEqualTo(propertyname="parameter", literal="Organische C - percentage"),
            PropertyIsEqualTo(propertyname="diepte_tot_cm", literal = '30'),
            PropertyIsEqualTo(propertyname="diepte_van_cm", literal = '0')
        ]))


bodemobservaties.head()

[000/066] ..................................................
[050/066] ................


,pkey_bodemobservatie,pkey_bodemlocatie,pkey_parent,x,y,mv_mtaw,diepte_van_cm,diepte_tot_cm,observatiedatum,invoerdatum,parametergroep,parameter,detectie,waarde,eenheid,veld_labo,methode,betrouwbaarheid
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,132594.0,193978.0,3.0,0.0,30.0,1952-04-22,NaN,Bodem_chemisch,Organische C - percentage,NaN,1.12142304717711,%,veld,Aardewerk oude methode organische koolstof,onbekend
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,102902.0,188823.0,8.0,0.0,30.0,1953-07-07,NaN,Bodem_chemisch,Organische C - percentage,NaN,1.430781129157,%,veld,Aardewerk oude methode organische koolstof,onbekend
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,133454.0,188789.0,6.0,0.0,30.0,1952-04-16,NaN,Bodem_chemisch,Organische C - percentage,NaN,0.587780355761794,%,veld,Aardewerk oude methode organische koolstof,onbekend
3,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,97220.0,185968.0,10.0,0.0,30.0,1962-08-23,NaN,Bodem_chemisch,Organische C - percentage,NaN,1.98,%,veld,Aardewerk nieuwe methode organische koolstof,onbekend
4,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,107945.0,207928.0,7.0,0.0,30.0,1952-08-06,NaN,Bodem_chemisch,Organische C - percentage,NaN,1.7092034029389,%,veld,Aardewerk oude methode organische koolstof,onbekend


Now we have all observations with the requested properties. 
Next we need to link them with the bodemlocatie

In [34]:
from pydov.search.bodemlocatie import BodemlocatieSearch
from pydov.util.query import Join
import pandas as pd

# Find bodemlocatie information for all observations
bodemlocatie = BodemlocatieSearch()
bodemlocaties = bodemlocatie.search(query=Join(bodemobservaties, on = 'pkey_bodemlocatie', using='pkey_bodemlocatie'))

# remove x, y, mv_mtaw from observatie dataframe to prevent duplicates while merging
bodemobservaties = bodemobservaties.drop(['x', 'y', 'mv_mtaw'], axis=1)

# Merge the bodemlocatie information together with the observation information
merged = pd.merge(bodemobservaties, bodemlocaties, on="pkey_bodemlocatie", how='left')

merged.head()

[000/066] ..................................................
[050/066] ................


,pkey_bodemobservatie,pkey_bodemlocatie,pkey_parent,diepte_van_cm,diepte_tot_cm,observatiedatum,invoerdatum_x,parametergroep,parameter,detectie,...,type,waarnemingsdatum,doel,x,y,mv_mtaw,erfgoed,bodemstreek,invoerdatum_y,educatieve_waarde
0,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,0.0,30.0,1952-04-22,NaN,Bodem_chemisch,Organische C - percentage,NaN,...,profielput,1952-04-22,bodemprofielen en oppervlaktemonsters karterin...,132594.0,193978.0,3.0,False,Zandstreek,2019-10-11,OK
1,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,0.0,30.0,1953-07-07,NaN,Bodem_chemisch,Organische C - percentage,NaN,...,profielput,1953-07-07,bodemprofielen en oppervlaktemonsters karterin...,102902.0,188823.0,8.0,False,Zandstreek,2019-10-11,OK
2,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,0.0,30.0,1952-04-16,NaN,Bodem_chemisch,Organische C - percentage,NaN,...,profielput,1952-04-16,bodemprofielen en oppervlaktemonsters karterin...,133454.0,188789.0,6.0,False,Zandstreek,2019-10-11,OK
3,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,0.0,30.0,1962-08-23,NaN,Bodem_chemisch,Organische C - percentage,NaN,...,profielput,1962-08-23,bodemprofielen en oppervlaktemonsters karterin...,97220.0,185968.0,10.0,False,Zandstreek,2019-10-11,OK
4,https://www.dov.vlaanderen.be/data/bodemobserv...,https://www.dov.vlaanderen.be/data/bodemlocati...,https://www.dov.vlaanderen.be/data/bodemmonste...,0.0,30.0,1952-08-06,NaN,Bodem_chemisch,Organische C - percentage,NaN,...,profielput,1952-08-06,bodemprofielen en oppervlaktemonsters karterin...,107945.0,207928.0,7.0,False,Zandstreek,2019-10-11,OK


To export the results to CSV, you can use for example: 
```python
merged.to_csv("test.csv")
```

We can plot also the results on a map
This can take some time!

In [35]:
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

#convert coordinates to wgs84
merged['lon'], merged['lat'] = zip(*map(convert_latlon, merged['x'], merged['y']))

# Get only location and value
loclist = merged[['lat', 'lon']].values.tolist()

# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[merged['lat'].mean(), merged['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    popup = 'Bodemlocatie: ' + merged['pkey_bodemlocatie'][loc] 
    popup = popup + '<br> Bodemobservatie: ' + merged['pkey_bodemobservatie'][loc]
    popup = popup + '<br> Value: ' + merged['waarde'][loc] + "%"
    folium.Marker(loclist[loc], popup=popup).add_to(marker_cluster)
fmap

### Calculate carbon stock in Ghent in the layer 0 - 23 cm

*At the moment, there are no bulkdensities available. As soon as there are observations with bulkdensities, this example can be used to calculate a carbon stock in a layer.*

Get boundaries of Ghent using WFS

In [36]:
from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService
from pydov.util.location import (
    GmlFilter,
    Within,
)

stadsgrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

gent_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Gent')
gent_poly = stadsgrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(gent_filter.toXML()).decode("utf8")).read()


First get all observations in Ghent for organisch C percentage in requested layer

In [37]:
from owslib.fes import PropertyIsEqualTo, PropertyIsGreaterThan, PropertyIsLessThan
from owslib.fes import And

from pydov.search.bodemobservatie import BodemobservatieSearch

bodemobservatie = BodemobservatieSearch()

# all layers intersect the layer 0-23cm
carbon_observaties = bodemobservatie.search(
        location=GmlFilter(gent_poly, Within),
        query=And([
            PropertyIsEqualTo(propertyname="parameter", literal="Organische C - percentage"),
            PropertyIsGreaterThan(propertyname="diepte_tot_cm", literal = '0'),
            PropertyIsLessThan(propertyname="diepte_van_cm", literal = '23')
        ]),
        return_fields=('pkey_bodemlocatie', 'waarde'))
carbon_observaties = carbon_observaties.rename(columns={"waarde": "organic_c_percentage"})
carbon_observaties.head()


,pkey_bodemlocatie,organic_c_percentage
0,https://www.dov.vlaanderen.be/data/bodemlocati...,1.24516627996906
1,https://www.dov.vlaanderen.be/data/bodemlocati...,0.889404485692189
2,https://www.dov.vlaanderen.be/data/bodemlocati...,2.43619489559165
3,https://www.dov.vlaanderen.be/data/bodemlocati...,1.10595514307811
4,https://www.dov.vlaanderen.be/data/bodemlocati...,1.430781129157


Then get all observations in Ghent for bulkdensity in requested layer

In [38]:
density_observaties = bodemobservatie.search(
        location=GmlFilter(gent_poly, Within),
        query=And([
            PropertyIsEqualTo(propertyname="parameter", literal="Bulkdensiteit - gemeten"),
            PropertyIsGreaterThan(propertyname="diepte_tot_cm", literal = '0'),
            PropertyIsLessThan(propertyname="diepte_van_cm", literal = '23')
        ]),
        return_fields=('pkey_bodemlocatie', 'waarde'))

density_observaties = density_observaties.rename(columns={"waarde": "bulkdensity"})
density_observaties.head()

,pkey_bodemlocatie,bulkdensity


Merge results together based on their bodemlocatie. Only remains the records where both parameters exists

In [39]:
import pandas as pd

merged = pd.merge(carbon_observaties, density_observaties, on="pkey_bodemlocatie")

merged.head()

,pkey_bodemlocatie,organic_c_percentage,bulkdensity
